In [1]:
import scipy.io
import pandas as pd
import matplotlib.pyplot as plt
import pickle

In [2]:
pathToAppearsTest = '/mnt/rhome/paa/pedestrian/attribute_analysis_pedestrian/attribute_analysis_pedestrian/RAPv2/RAPv2_test0.csv'
pathToAppearsVal = '/mnt/rhome/paa/pedestrian/attribute_analysis_pedestrian/attribute_analysis_pedestrian/RAPv2/RAPv2_val0.csv'
pathToAppearsTrain = '/mnt/rhome/paa/pedestrian/attribute_analysis_pedestrian/attribute_analysis_pedestrian/RAPv2/RAPv2_train0.csv'

pathToMaTest = '/mnt/rhome/paa/pedestrian/attribute_analysis_pedestrian/attribute_analysis_pedestrian/RAPv2/RAPv2_testing_ma.csv'
pathToF1Test = '/mnt/rhome/paa/pedestrian/attribute_analysis_pedestrian/attribute_analysis_pedestrian/RAPv2/RAPv2_testing_f1.csv'
pathToAccTest = '/mnt/rhome/paa/pedestrian/attribute_analysis_pedestrian/attribute_analysis_pedestrian/RAPv2/RAPv2_testing_acc.csv'
pathToPrecTest = '/mnt/rhome/paa/pedestrian/attribute_analysis_pedestrian/attribute_analysis_pedestrian/RAPv2/RAPv2_testing_prec.csv'

pathToMaTraining= '/mnt/rhome/paa/pedestrian/attribute_analysis_pedestrian/attribute_analysis_pedestrian/RAPv2/RAPv2_training_ma.csv'
pathToF1Training = '/mnt/rhome/paa/pedestrian/attribute_analysis_pedestrian/attribute_analysis_pedestrian/RAPv2/RAPv2_training_f1.csv'
pathToAccTraining = '/mnt/rhome/paa/pedestrian/attribute_analysis_pedestrian/attribute_analysis_pedestrian/RAPv2/RAPv2_training_acc.csv'
pathToPrecTraining = '/mnt/rhome/paa/pedestrian/attribute_analysis_pedestrian/attribute_analysis_pedestrian/RAPv2/RAPv2_training_prec.csv'

In [3]:
test_dt = pd.read_csv(pathToAppearsTest, sep='\t', index_col=False)
test_dt_ma = pd.read_csv(pathToMaTest, sep=',', index_col=False)
test_dt_f1 = pd.read_csv(pathToF1Test, sep=',', index_col=False)
test_dt_acc = pd.read_csv(pathToAccTest, sep=',', index_col=False)
test_dt_prec = pd.read_csv(pathToPrecTest, sep=',', index_col=False)

In [4]:
def getTestingCorrelation(test_dt,test_dt_ma,test_dt_f1,test_dt_acc,test_dt_prec):
    notColumn = 'Unnamed: 0'
    attributes = list(test_dt.columns)
    attributes.remove(notColumn)

    seed='605'
    bestEpoch = 8

    metrics_attributes_ma = [ 'testing_' + 'SkirtDress'+ '_ma_' + seed if attribute == 'Skirt&Dress' else 'testing_' + attribute + '_ma_' + seed for attribute in attributes]
    metrics_attributes_f1 = [ 'testing_' + 'SkirtDress'+ '_label_f1_' + seed if attribute == 'Skirt&Dress' else 'testing_' + attribute + '_label_f1_' + seed for attribute in attributes]
    metrics_attributes_acc = [ 'testing_' + 'SkirtDress'+ '_label_acc_' + seed if attribute == 'Skirt&Dress' else 'testing_' + attribute + '_label_acc_' + seed for attribute in attributes]
    metrics_attributes_prec = [ 'testing_' + 'SkirtDress'+ '_label_prec_' + seed if attribute == 'Skirt&Dress' else 'testing_' + attribute + '_label_prec_' + seed for attribute in attributes]


    pattern_ma = '|'.join([f'(?i){attribute}' for attribute in metrics_attributes_ma])
    pattern_f1 = '|'.join([f'(?i){attribute}' for attribute in metrics_attributes_f1])
    pattern_acc = '|'.join([f'(?i){attribute}' for attribute in metrics_attributes_acc])
    pattern_prec = '|'.join([f'(?i){attribute}' for attribute in metrics_attributes_prec])

    dt_ma_values = test_dt_ma[(test_dt_ma['step']==bestEpoch) & (test_dt_ma['key'].str.contains(pattern_ma))][['key','value']]
    dt_f1_values = test_dt_f1[(test_dt_f1['step']==bestEpoch) & (test_dt_f1['key'].str.contains(pattern_f1))][['key','value']]
    dt_acc_values = test_dt_acc[(test_dt_acc['step']==bestEpoch) & (test_dt_acc['key'].str.contains(pattern_acc))][['key','value']]
    dt_prec_values = test_dt_prec[(test_dt_prec['step']==bestEpoch) & (test_dt_prec['key'].str.contains(pattern_prec))][['key','value']]

    values_representation = list(test_dt[attributes].sum())
    # values representation is aligned with:
    #metrics_attributes_ma
    #metrics_attributes_f1
    #metrics_attributes_prec
    #metrics_attributes_acc

    values_representation_ma = [ values_representation[metrics_attributes_ma.index(key)] for key in dt_ma_values['key'].unique()]
    values_representation_f1 = [ values_representation[metrics_attributes_f1.index(key)] for key in dt_f1_values['key'].unique()]
    values_representation_prec = [ values_representation[metrics_attributes_prec.index(key)] for key in dt_prec_values['key'].unique()]
    values_representation_acc = [ values_representation[metrics_attributes_acc.index(key)] for key in dt_acc_values['key'].unique()]

    dt_ma_values['representation'] = values_representation_ma
    dt_f1_values['representation'] = values_representation_f1
    dt_prec_values['representation'] = values_representation_prec
    dt_acc_values['representation'] = values_representation_acc

    dt_ma_values['value'] = pd.to_numeric(dt_ma_values['value'])
    dt_f1_values['value'] = pd.to_numeric(dt_f1_values['value'])
    dt_prec_values['value'] = pd.to_numeric(dt_prec_values['value'])
    dt_acc_values['value'] = pd.to_numeric(dt_acc_values['value'])

    print("Corr ma: ")
    print(dt_ma_values['value'].corr(dt_ma_values['representation']))

    print("Corr f1: ")
    print(dt_f1_values['value'].corr(dt_f1_values['representation']))

    print("Corr prec: ")
    print(dt_prec_values['value'].corr(dt_prec_values['representation']))

    print("Corr acc: ")
    print(dt_acc_values['value'].corr(dt_acc_values['representation']))

In [5]:
def getTrainingCorrelation(test_dt,test_dt_ma,test_dt_f1,test_dt_acc,test_dt_prec):
    notColumn = 'Unnamed: 0'
    attributes = list(test_dt.columns)
    attributes.remove(notColumn)

    seed='605'
    bestEpoch = 8

    metrics_attributes_ma = [ 'training_' + 'SkirtDress'+ '_ma_' + seed if attribute == 'Skirt&Dress' else 'training_' + attribute + '_ma_' + seed for attribute in attributes]
    metrics_attributes_f1 = [ 'training_' + 'SkirtDress'+ '_label_f1_' + seed if attribute == 'Skirt&Dress' else 'training_' + attribute + '_label_f1_' + seed for attribute in attributes]
    metrics_attributes_acc = [ 'training_' + 'SkirtDress'+ '_label_acc_' + seed if attribute == 'Skirt&Dress' else 'training_' + attribute + '_label_acc_' + seed for attribute in attributes]
    metrics_attributes_prec = [ 'training_' + 'SkirtDress'+ '_label_prec_' + seed if attribute == 'Skirt&Dress' else 'training_' + attribute + '_label_prec_' + seed for attribute in attributes]


    pattern_ma = '|'.join([f'(?i){attribute}' for attribute in metrics_attributes_ma])
    pattern_f1 = '|'.join([f'(?i){attribute}' for attribute in metrics_attributes_f1])
    pattern_acc = '|'.join([f'(?i){attribute}' for attribute in metrics_attributes_acc])
    pattern_prec = '|'.join([f'(?i){attribute}' for attribute in metrics_attributes_prec])

    dt_ma_values = test_dt_ma[(test_dt_ma['step']==bestEpoch) & (test_dt_ma['key'].str.contains(pattern_ma))][['key','value']]
    dt_f1_values = test_dt_f1[(test_dt_f1['step']==bestEpoch) & (test_dt_f1['key'].str.contains(pattern_f1))][['key','value']]
    dt_acc_values = test_dt_acc[(test_dt_acc['step']==bestEpoch) & (test_dt_acc['key'].str.contains(pattern_acc))][['key','value']]
    dt_prec_values = test_dt_prec[(test_dt_prec['step']==bestEpoch) & (test_dt_prec['key'].str.contains(pattern_prec))][['key','value']]

    values_representation = list(test_dt[attributes].sum())
    # values representation is aligned with:
    #metrics_attributes_ma
    #metrics_attributes_f1
    #metrics_attributes_prec
    #metrics_attributes_acc

    values_representation_ma = [ values_representation[metrics_attributes_ma.index(key)] for key in dt_ma_values['key'].unique()]
    values_representation_f1 = [ values_representation[metrics_attributes_f1.index(key)] for key in dt_f1_values['key'].unique()]
    values_representation_prec = [ values_representation[metrics_attributes_prec.index(key)] for key in dt_prec_values['key'].unique()]
    values_representation_acc = [ values_representation[metrics_attributes_acc.index(key)] for key in dt_acc_values['key'].unique()]

    dt_ma_values['representation'] = values_representation_ma
    dt_f1_values['representation'] = values_representation_f1
    dt_prec_values['representation'] = values_representation_prec
    dt_acc_values['representation'] = values_representation_acc

    dt_ma_values['value'] = pd.to_numeric(dt_ma_values['value'])
    dt_f1_values['value'] = pd.to_numeric(dt_f1_values['value'])
    dt_prec_values['value'] = pd.to_numeric(dt_prec_values['value'])
    dt_acc_values['value'] = pd.to_numeric(dt_acc_values['value'])

    print("Corr ma: ")
    print(dt_ma_values['value'].corr(dt_ma_values['representation']))

    print("Corr f1: ")
    print(dt_f1_values['value'].corr(dt_f1_values['representation']))

    print("Corr prec: ")
    print(dt_prec_values['value'].corr(dt_prec_values['representation']))

    print("Corr acc: ")
    print(dt_acc_values['value'].corr(dt_acc_values['representation']))

In [6]:
getTestingCorrelation(test_dt,test_dt_ma,test_dt_f1,test_dt_acc,test_dt_prec)

Corr ma: 
0.0031796712385235
Corr f1: 
0.5249763446310312
Corr prec: 
0.5779980814592606
Corr acc: 
0.6132490946450196


In [7]:
train_dt = pd.read_csv(pathToAppearsTrain, sep='\t', index_col=False)
train_dt_ma = pd.read_csv(pathToMaTraining, sep=',', index_col=False)
train_dt_f1 = pd.read_csv(pathToF1Training, sep=',', index_col=False)
train_dt_acc = pd.read_csv(pathToAccTraining, sep=',', index_col=False)
train_dt_prec = pd.read_csv(pathToPrecTraining, sep=',', index_col=False)

In [8]:
getTrainingCorrelation(train_dt, train_dt_ma, train_dt_f1, train_dt_acc, train_dt_prec)

Corr ma: 
0.012392457633730156
Corr f1: 
0.42634135384264793
Corr prec: 
0.44713358069348275
Corr acc: 
0.4905201331763452


In [16]:
test_dt.columns

Index(['Unnamed: 0', 'Femal', 'AgeLess16', 'Age17-30', 'Age31-45', 'Age46-60',
       'AgeBiger60', 'BodyFatter', 'BodyFat', 'BodyNormal',
       ...
       'attachment2-position-w', 'attachment2-position-h',
       'attachment3-position-x', 'attachment3-position-y',
       'attachment3-position-w', 'attachment3-position-h',
       'attachment4-position-x', 'attachment4-position-y',
       'attachment4-position-w', 'attachment4-position-h'],
      dtype='object', length=153)

In [11]:
def getTestingCorrelation(test_dt,test_dt_ma,test_dt_f1,test_dt_acc,test_dt_prec):
    notColumn = 'Unnamed: 0'
    attributes = list(test_dt.columns)
    attributes.remove(notColumn)

    seed='605'
    bestEpoch = 8

    metrics_attributes_ma = [ 'testing_' + 'SkirtDress'+ '_ma_' + seed if attribute == 'Skirt&Dress' else 'testing_' + attribute + '_ma_' + seed for attribute in attributes]
    metrics_attributes_f1 = [ 'testing_' + 'SkirtDress'+ '_label_f1_' + seed if attribute == 'Skirt&Dress' else 'testing_' + attribute + '_label_f1_' + seed for attribute in attributes]
    metrics_attributes_acc = [ 'testing_' + 'SkirtDress'+ '_label_acc_' + seed if attribute == 'Skirt&Dress' else 'testing_' + attribute + '_label_acc_' + seed for attribute in attributes]
    metrics_attributes_prec = [ 'testing_' + 'SkirtDress'+ '_label_prec_' + seed if attribute == 'Skirt&Dress' else 'testing_' + attribute + '_label_prec_' + seed for attribute in attributes]


    pattern_ma = '|'.join([f'(?i){attribute}' for attribute in metrics_attributes_ma])
    pattern_f1 = '|'.join([f'(?i){attribute}' for attribute in metrics_attributes_f1])
    pattern_acc = '|'.join([f'(?i){attribute}' for attribute in metrics_attributes_acc])
    pattern_prec = '|'.join([f'(?i){attribute}' for attribute in metrics_attributes_prec])

    dt_ma_values = test_dt_ma[(test_dt_ma['step']==bestEpoch) & (test_dt_ma['key'].str.contains(pattern_ma))][['key','value']]
    dt_f1_values = test_dt_f1[(test_dt_f1['step']==bestEpoch) & (test_dt_f1['key'].str.contains(pattern_f1))][['key','value']]
    dt_acc_values = test_dt_acc[(test_dt_acc['step']==bestEpoch) & (test_dt_acc['key'].str.contains(pattern_acc))][['key','value']]
    dt_prec_values = test_dt_prec[(test_dt_prec['step']==bestEpoch) & (test_dt_prec['key'].str.contains(pattern_prec))][['key','value']]

    values_representation = list(test_dt[attributes].sum())
    # values representation is aligned with:
    #metrics_attributes_ma
    #metrics_attributes_f1
    #metrics_attributes_prec
    #metrics_attributes_acc

    values_representation_ma = [ values_representation[metrics_attributes_ma.index(key)] for key in dt_ma_values['key'].unique()]
    values_representation_f1 = [ values_representation[metrics_attributes_f1.index(key)] for key in dt_f1_values['key'].unique()]
    values_representation_prec = [ values_representation[metrics_attributes_prec.index(key)] for key in dt_prec_values['key'].unique()]
    values_representation_acc = [ values_representation[metrics_attributes_acc.index(key)] for key in dt_acc_values['key'].unique()]

    dt_ma_values['representation'] = values_representation_ma
    dt_f1_values['representation'] = values_representation_f1
    dt_prec_values['representation'] = values_representation_prec
    dt_acc_values['representation'] = values_representation_acc

    dt_ma_values['value'] = pd.to_numeric(dt_ma_values['value'])
    dt_f1_values['value'] = pd.to_numeric(dt_f1_values['value'])
    dt_prec_values['value'] = pd.to_numeric(dt_prec_values['value'])
    dt_acc_values['value'] = pd.to_numeric(dt_acc_values['value'])

    print("Corr ma: ")
    print(dt_ma_values['value'].corr(dt_ma_values['representation']))

    print("Corr f1: ")
    print(dt_f1_values['value'].corr(dt_f1_values['representation']))

    print("Corr prec: ")
    print(dt_prec_values['value'].corr(dt_prec_values['representation']))

    print("Corr acc: ")
    print(dt_acc_values['value'].corr(dt_acc_values['representation']))

In [75]:
def getTestingWithTrainingRepresentationCorrelation(test_dt,test_dt_ma,test_dt_f1,test_dt_acc,test_dt_prec,attributes):
    notColumn = 'Unnamed: 0'
    #attributes = list(test_dt.columns)
    #attributes.remove(notColumn)

    seed='605'
    bestEpoch = 8

    metrics_attributes_ma = [attribute + '_ma_' + seed for attribute in attributes]
    metrics_attributes_f1 =  [attribute + '_label_f1_' + seed for attribute in attributes]
    metrics_attributes_acc = [attribute + '_label_acc_' + seed for attribute in attributes]
    metrics_attributes_prec = [attribute + '_label_prec_' + seed for attribute in attributes]

    print(len(metrics_attributes_ma))

    pattern_ma = '|'.join([f'(?i){attribute}' for attribute in metrics_attributes_ma])
    pattern_f1 = '|'.join([f'(?i){attribute}' for attribute in metrics_attributes_f1])
    pattern_acc = '|'.join([f'(?i){attribute}' for attribute in metrics_attributes_acc])
    pattern_prec = '|'.join([f'(?i){attribute}' for attribute in metrics_attributes_prec])

    test_dt_ma.sort_values('key',inplace=True)
    test_dt_f1.sort_values('key',inplace=True)
    test_dt_acc.sort_values('key',inplace=True)
    test_dt_prec.sort_values('key',inplace=True)

    dt_ma_values = test_dt_ma[(test_dt_ma['step']==bestEpoch) & (test_dt_ma['key'].str.contains(pattern_ma))][['key','value']]
    dt_f1_values = test_dt_f1[(test_dt_f1['step']==bestEpoch) & (test_dt_f1['key'].str.contains(pattern_f1))][['key','value']]
    dt_acc_values = test_dt_acc[(test_dt_acc['step']==bestEpoch) & (test_dt_acc['key'].str.contains(pattern_acc))][['key','value']]
    dt_prec_values = test_dt_prec[(test_dt_prec['step']==bestEpoch) & (test_dt_prec['key'].str.contains(pattern_prec))][['key','value']]
    
    print(len(dt_ma_values))
    print(len(attributes))

    metrics_attributes_ma=sorted(metrics_attributes_ma)
    metrics_attributes_f1=sorted(metrics_attributes_f1)
    metrics_attributes_acc=sorted(metrics_attributes_acc)
    metrics_attributes_prec=sorted(metrics_attributes_prec)

    attributes2=attributes
    attributes2 =sorted(attributes2)

    values_representation = list(test_dt[attributes2].sum())
    # values representation is aligned with:
    #metrics_attributes_ma
    #metrics_attributes_f1
    #metrics_attributes_prec
    #metrics_attributes_acc

    values_representation_ma = [ values_representation[i] for i in range(0,len(attributes))]
    values_representation_f1 = [ values_representation[i] for i in range(0,len(attributes))]
    values_representation_prec = [ values_representation[i] for i in range(0,len(attributes))]
    values_representation_acc = [ values_representation[i] for i in range(0,len(attributes))]

    print(len(values_representation_ma))

    dt_ma_values['representation'] = values_representation_ma
    dt_f1_values['representation'] = values_representation_f1
    dt_prec_values['representation'] = values_representation_prec
    dt_acc_values['representation'] = values_representation_acc

    dt_ma_values['value'] = pd.to_numeric(dt_ma_values['value'])
    dt_f1_values['value'] = pd.to_numeric(dt_f1_values['value'])
    dt_prec_values['value'] = pd.to_numeric(dt_prec_values['value'])
    dt_acc_values['value'] = pd.to_numeric(dt_acc_values['value'])

    print("Corr ma: ")
    print(dt_ma_values['value'].corr(dt_ma_values['representation']))

    print("Corr f1: ")
    print(dt_f1_values['value'].corr(dt_f1_values['representation']))

    print("Corr prec: ")
    print(dt_prec_values['value'].corr(dt_prec_values['representation']))

    print("Corr acc: ")
    print(dt_acc_values['value'].corr(dt_acc_values['representation']))

In [76]:
listAttributesRAPv2 = ['hs-BaldHead', 'hs-LongHair', 'hs-BlackHair', 'hs-Hat', 'hs-Glasses','ub-Shirt','ub-Sweater','ub-Vest','ub-TShirt','ub-Cotton','ub-Jacket','ub-SuitUp','ub-Tight','ub-ShortSleeve','ub-Others','lb-LongTrousers','lb-Skirt','lb-ShortSkirt','lb-Dress','lb-Jeans','lb-TightTrousers','shoes-Leather', 'shoes-Sports', 'shoes-Boots', 'shoes-Cloth', 'shoes-Casual', 'shoes-Other','attachment-Backpack','attachment-ShoulderBag','attachment-HandBag','attachment-Box','attachment-PlasticBag','attachment-PaperBag','attachment-HandTrunk','attachment-Other','AgeLess16', 'Age17-30', 'Age31-45', 'Age46-60','Femal','BodyFat','BodyNormal','BodyThin','Customer','Employee','action-Calling','action-Talking','action-Gathering','action-Holding','action-Pushing','action-Pulling','action-CarryingByArm','action-CarryingByHand','action-Other']
listAttributesRAPzs = listAttributesRAPv2
listAttributesRAPzs.remove('Age46-60')
len(listAttributesRAPzs)

53

In [77]:
getTestingWithTrainingRepresentationCorrelation(train_dt,test_dt_ma,test_dt_f1,test_dt_acc,test_dt_prec,listAttributesRAPzs)

53
52
53
53


ValueError: Length of values (53) does not match length of index (52)